In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
import cv2

def cmp(a,b):  
    bBlur = cv2.GaussianBlur(b,(5,5),0)
    aBlur = cv2.GaussianBlur(a,(5,5),0)
    
    aBlurGray = cv2.cvtColor(aBlur, cv2.COLOR_BGR2GRAY)
    bBlurGray = cv2.cvtColor(bBlur, cv2.COLOR_BGR2GRAY)
    
    frameDelta = cv2.absdiff(bBlurGray, aBlurGray)
    
    _, thresh = cv2.threshold(frameDelta, 40, 255, cv2.THRESH_BINARY)
    thresh = cv2.dilate(thresh, None, iterations=2)
    thresh = cv2.cvtColor(thresh,cv2.COLOR_GRAY2RGB)
    thresh[np.where((thresh==[255, 255, 255]).all(axis=2))] = [0,0,255]
    
    image_out = cv2.addWeighted(thresh, 0.5, a, 1, 0)
    
    cv2.putText(image_out, 'Red Light', (450, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    cv2.imshow('moving detector',image_out)
    thresh = cv2.cvtColor(thresh,cv2.COLOR_RGB2GRAY)
    
    if cv2.countNonZero(thresh) == 0:
        return 0
    else:
        return 1

cap = cv2.VideoCapture(0)
ret, after = cap.read()
before, sgn = after, False
cur = time.time()

while True:
    
    ret, after = cap.read()
    key = cv2.waitKey(20) & 0xff
    #cv2.imshow('video', after)
    img = np.zeros((300,300,3), np.uint8)
    
    if sgn:
        if(cmp(before,after)==0):
            img[:,:]=(0,255,0)
            cv2.imshow('map', img)
        else:
            img[:,:]=(0,0, 255)
            cv2.imshow('map', img)
        before = after
    else:
        before = after
        show = after.copy()
        cv2.putText(show, 'Green Light', (440, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('moving detector',show)
        img[:,:]=(0,255,0)
        cv2.imshow('map', img)

    if time.time() - cur >= 5:
        cur = time.time()
        sgn = not(sgn)
        
    if key == 27: # Esc
        break
        
cv2.destroyAllWindows()
cap.release()